# Commonsense Causal Reasoning

In [20]:
import torch
import numpy as np
from dataclasses import dataclass
from typing import Optional, Union
from transformers import AutoTokenizer
from datasets import load_dataset, load_metric
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy

## Load COPA Dataset

In [2]:
copa = load_dataset("super_glue", "copa")

Found cached dataset super_glue (/home/fyy/.cache/huggingface/datasets/super_glue/copa/1.0.3/bb9675f958ebfee0d5d6dc5476fafe38c79123727a7258d515c450873dbdbbed)


  0%|          | 0/3 [00:00<?, ?it/s]

In [24]:
# See one example

copa["train"][0]

{'premise': 'My body cast a shadow over the grass.',
 'choice1': 'The sun was rising.',
 'choice2': 'The grass was cut.',
 'question': 'cause',
 'idx': 0,
 'label': 0}

## Preprocessing

See [Multiple choice](https://huggingface.co/docs/transformers/tasks/multiple_choice)

In [4]:
def preprocess_function(examples):
    first_sentences = [[context]*2 for context in examples["premise"]]
    first_sentences = sum(first_sentences, [])
    question_headers = examples["question"]
    second_sentences = [
        [f"because {examples[end][i]}" for end in ["choice1", "choice2"]] if header == "cause" else\
        [f"so {examples[end][i]}" for end in ["choice1", "choice2"]]\
            for i, header in enumerate(question_headers)
    ]
    second_sentences = sum(second_sentences, [])
    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    return {k: [v[i : i + 2] for i in range(0, len(v), 2)] for k, v in tokenized_examples.items()}

In [5]:
tokenized_copa = copa.map(preprocess_function, batched=True)

Parameter 'function'=<function preprocess_function at 0x7fca0a3757a0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [6]:
@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [7]:
model = AutoModelForMultipleChoice.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly

## Compute Metrics

In [18]:
def compute_metrics(eval_pred):
    metric = load_metric("super_glue", "copa")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

## Training and Evaluation

In [21]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_copa["train"],
    eval_dataset=tokenized_copa["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    compute_metrics = compute_metrics
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: choice2, question, choice1, idx, premise. If choice2, question, choice1, idx, premise are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
/home/fyy/miniconda3/envs/py37/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 400
  Num Epochs =

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.433309,0.560000
2,No log,2.471225,0.540000
3,No log,2.836496,0.560000


The following columns in the evaluation set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: choice2, question, choice1, idx, premise. If choice2, question, choice1, idx, premise are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 100
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: choice2, question, choice1, idx, premise. If choice2, question, choice1, idx, premise are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 100
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: choice2, question, choice1, idx, premise. If choice2, question, choice1, idx, premise are not 

TrainOutput(global_step=75, training_loss=0.0005427950620651245, metrics={'train_runtime': 17.0008, 'train_samples_per_second': 70.585, 'train_steps_per_second': 4.412, 'total_flos': 29681949428160.0, 'train_loss': 0.0005427950620651245, 'epoch': 3.0})